# Making the 0.792 model : 

### Making the model and Inferencing steps:

<ul>  
    <li>Using the datasets : chaii, mlqa, squad, tamil_xquad </li>
    <li>Training for 2 epochs : <a href="https://www.kaggle.com/kishalmandal/chaii-fit-2-epochs-mlqa-xquad-chaii/">chaii | FIT - 2 epochs | mlqa, xquad, chaii</a> </li>
    <li>Training for 7 epochs with tamil_xquad: <a href="https://www.kaggle.com/kishalmandal/chaii-fit-7-epochs-extra-tamil-data/">chaii | FIT - 7 epochs | Extra Tamil Data</a> </li>
    <li>Inferencing from 5 folds | fold-0 and fold-1 (7-epochs) | fold-2, fold-3 and fold-4 (2-epochs)| based on cross validation scores and a little bit of experimentation 😜: <a href="https://www.kaggle.com/kishalmandal/5-epochs-infer-combined-model-0-792/">5 epochs | INFER | combined model (0.792)</a></li>
    
    
</ul>

### References : 

<ul> 
    <li>For training: <a href="https://www.kaggle.com/rhtsingh/chaii-qa-5-fold-xlmroberta-torch-fit">chaii QA - 5 Fold XLMRoberta Torch | FIT</a> by <a href="https://www.kaggle.com/rhtsingh">torch</a></li>
    <li>For Inference: <a href="https://www.kaggle.com/rhtsingh/chaii-qa-5-fold-xlmroberta-torch-infer">chaii QA - 5 Fold XLMRoberta Torch | Infer</a> by <a href="https://www.kaggle.com/rhtsingh">torch</a></li>
    <li>For post processing outputs: <a href="https://www.kaggle.com/nbroad/chaii-qa-torch-5-fold-with-post-processing-765">chaii QA-Torch 5 fold with post-processing (.765)</a> by <a href="https://www.kaggle.com/nbroad">Nicholas Broad 🟢</a></li>
</ul>


### Important Notes:

<ul>
    <li>Some one experimented with the max_seq_length and doc_stride to bring my previous 0.785 to 0.792. Then I performed some more experiments and saw more than 400 decreases the score (~0.3-0.4) while decreasing below 400 upto around 384 does not change the score by much (~0.2)</li>
    <li>Vigourous change or lr and decay rates reduces the performance.</li>
    <li>I took weighted average based on the validation loss increased score by (~0.01)</li>
</ul>

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import gc
gc.enable()
import math
import json
import time
import random
import multiprocessing
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from sklearn import model_selection
from string import punctuation

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
from torch.utils.data.distributed import DistributedSampler

try:
    from apex import amp
    APEX_INSTALLED = True
except ImportError:
    APEX_INSTALLED = False

import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    logging,
    MODEL_FOR_QUESTION_ANSWERING_MAPPING,
)
logging.set_verbosity_warning()
logging.set_verbosity_error()

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def optimal_num_of_loader_workers():
    num_cpus = multiprocessing.cpu_count()
    num_gpus = torch.cuda.device_count()
    optimal_value = min(num_cpus, num_gpus*4) if num_gpus else num_cpus - 1
    return optimal_value

print(f"Apex AMP Installed :: {APEX_INSTALLED}")
MODEL_CONFIG_CLASSES = list(MODEL_FOR_QUESTION_ANSWERING_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

2021-11-15 05:42:31.898676: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Apex AMP Installed :: False


In [2]:
class Config:
    # model
    model_type = 'xlm_roberta'
    model_name_or_path = "../input/5foldsroberta/output/checkpoint-fold-0"#"../input/xlm-roberta-large-squad-v2"
    config_name = "../input/5foldsroberta/output/checkpoint-fold-0"#"../input/xlm-roberta-large-squad-v2"
    fp16 = True if APEX_INSTALLED else False
    fp16_opt_level = "O1"
    gradient_accumulation_steps = 2

    # tokenizer
    tokenizer_name = "../input/5foldsroberta/output/checkpoint-fold-0"#"../input/xlm-roberta-large-squad-v2"
    max_seq_length = 400
    doc_stride = 135

    # train
    epochs = 1
    train_batch_size = 4
    eval_batch_size = 128

    # optimzer
    optimizer_type = 'AdamW'
    learning_rate = 1e-5
    weight_decay = 1e-2
    epsilon = 1e-8
    max_grad_norm = 1.0

    # scheduler
    decay_name = 'linear-warmup'
    warmup_ratio = 0.1

    # logging
    logging_steps = 10

    # evaluate
    output_dir = 'output'
    seed = 2021

In [3]:
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

In [4]:
class Model(nn.Module):
    def __init__(self, modelname_or_path, config):
        super(Model, self).__init__()
        self.config = config
        self.xlm_roberta = AutoModel.from_pretrained(modelname_or_path, config=config)
        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self._init_weights(self.qa_outputs)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()

    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
    ):
        outputs = self.xlm_roberta(
            input_ids,
            attention_mask=attention_mask,
        )

        sequence_output = outputs[0]
        pooled_output = outputs[1]
        
        # sequence_output = self.dropout(sequence_output)
        qa_logits = self.qa_outputs(sequence_output)
        
        start_logits, end_logits = qa_logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
    
        return start_logits, end_logits

In [5]:
def make_model(args):
    config = AutoConfig.from_pretrained(args.config_name)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name)
    model = Model(args.model_name_or_path, config=config)
    return config, tokenizer, model

In [6]:
def prepare_test_features(args, example, tokenizer):
    example["question"] = example["question"].lstrip()
    
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=args.max_seq_length,
        stride=args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example['id']
        feature['context'] = example['context']
        feature['question'] = example['question']
        feature['input_ids'] = tokenized_example['input_ids'][i]
        feature['attention_mask'] = tokenized_example['attention_mask'][i]
        feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
        feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)
    return features

# Combining different data

In [7]:
test0 = pd.read_csv('../input/mlqa-hindi-processed/mlqa_hindi.csv')
print(len(test0))
test0.head()

5425


,context,question,answer_text,answer_start,language
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi


In [8]:
test1 = pd.read_csv('../input/mlqa-hindi-processed/xquad.csv')
print(len(test1))
test1.head()

1190


,context,question,answer_text,answer_start,language
0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35,hindi
1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380,hindi
2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743,hindi
3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90,hindi
4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169,hindi


In [9]:
test2 = pd.read_csv('../input/squad-translated-to-tamil-for-chaii/squad_translated_tamil.csv')
print(len(test2))
test2.loc[:,'language']='tamil'
test2.head()

3567


,context,question,answer_text,answer_start,language
0,"மாகாணமானது பாரம்பரியமாக ""மீன் மற்றும் அரிசி நி...",மாகாணத்தின் இரண்டாவது முக்கிய பயிர் என்ன?,கோதுமை,133.0,tamil
1,Gregorian சீர்திருத்தம் இரண்டு பகுதிகளைக் கொண்...,சந்திர காலண்டரைப் பயன்படுத்தி என்ன கணக்கிடப்பட...,ஈஸ்டர் தேதி,219.0,tamil
2,அ. டோங்க்கோபா மிங் நீதிமன்றத்தில் தோன்றும் மறு...,மிங் நீதிமன்றம் சாங்க்கபாவிற்கு இரண்டாவது கோரி...,1413,352.0,tamil
3,"1919 ஆம் ஆண்டில், ஐரிஷ் ஹோம் ஆட்சிக்கு தாமதங்க...",அயர்லாந்தின் எத்தனை மாவட்டங்கள் வட அயர்லாந்தில...,ஆறு,816.0,tamil
4,அந்த யுத்தம் தவிர்க்க முடியாதது என்பதை உணர்ந்த...,ஆஸ்திரியா என்ன பிராந்தியத்தை திரும்பப் பெற முட...,ஆஸ்திரியா பிரான்சுடன் ஒரு கூட்டணியை உருவாக்கிய...,213.0,tamil


In [10]:
# result = pd.concat([test0,test1,test2])
result = pd.concat([test1,test2[:1500]])
print(len(result))
result['answer_start']= result['answer_start'].astype(float).astype(int)
id_column_value=[]
for i in range(len(result)):
    id_column_value.append(str(i))
result['id']=id_column_value
result.head()

2690


,context,question,answer_text,answer_start,language,id
0,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,पैंथर्स डिफ़ेंस ने कितने अंक दिए?,308,35,hindi,0
1,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जेरेड एलन के पास कितने करियर सैक थे?,136,380,hindi,1
2,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,ल्यूक कुएक्ली ने कितने टैकल रजिस्टर किए?,118,743,hindi,2
3,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,जोश नॉर्मन ने कितने बॉल को इंटरसेप्ट किया?,चार,90,hindi,3
4,पैंथर्स की डिफ़ेन्स ने लीग में केवल 308 अंक दिए...,इस सीज़न में टीम से किसने सबसे अधिक सैक रजिस्टर...,कावन शॉर्ट,169,hindi,4


# Processing data

In [11]:
# test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')#[:1]
# test = pd.concat([test,result[:1]])
test = pd.concat([test,result])

test['context'] = test['context'].apply(lambda x: ' '.join(x.split()))
test['question'] = test['question'].apply(lambda x: ' '.join(x.split()))

#base_model_path = '../input/chaii-qa-5-fold-xlmroberta-torch-fit'

tokenizer = AutoTokenizer.from_pretrained(Config().tokenizer_name)

test_features = []
for i, row in test.iterrows():
    test_features += prepare_test_features(Config(), row, tokenizer)

args = Config()
test_dataset = DatasetRetriever(test_features, mode='test')
test_dataloader = DataLoader(
    test_dataset,
    batch_size=args.eval_batch_size, 
    sampler=SequentialSampler(test_dataset),
    num_workers=optimal_num_of_loader_workers(),
    pin_memory=True, 
    drop_last=False
)

In [12]:
len(test)

3804

In [13]:
base_model = '../input/5foldsroberta/output/'

In [14]:
def get_predictions(checkpoint_path):
    config, tokenizer, model = make_model(Config())
    model.cuda();
    model.load_state_dict(
        torch.load(checkpoint_path)
    );
    
    start_logits = []
    end_logits = []
    for batch in test_dataloader:
        with torch.no_grad():
            outputs_start, outputs_end = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(outputs_start.cpu().numpy().tolist())
            end_logits.append(outputs_end.cpu().numpy().tolist())
            del outputs_start, outputs_end
    del model, tokenizer, config
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [15]:
start_logits1, end_logits1 = get_predictions('../input/5foldsroberta/output/checkpoint-fold-0/pytorch_model.bin')
print('pred1 complete')
start_logits2, end_logits2 = get_predictions('../input/5foldsroberta/output/checkpoint-fold-1/pytorch_model.bin')
print('pred2 complete')
start_logits3, end_logits3 = get_predictions('../input/5foldsroberta/output/checkpoint-fold-2/pytorch_model.bin')
print('pred3 complete')
start_logits4, end_logits4 = get_predictions('../input/5foldsroberta/output/checkpoint-fold-3/pytorch_model.bin')
print('pred4 complete')
start_logits5, end_logits5 = get_predictions('../input/5foldsroberta/output/checkpoint-fold-4/pytorch_model.bin')
print('pred5 complete')




start_logits = (start_logits1 + start_logits2 + start_logits3 +start_logits4+ start_logits5 )/5
end_logits = (end_logits1 + end_logits2 + end_logits3 +end_logits4 + end_logits5)/5

pred1 complete
pred2 complete
pred3 complete
pred4 complete
pred5 complete


In [16]:
# normalize
normalizer = torch.nn.Softmax(dim=1)

start_logits = normalizer(torch.from_numpy(start_logits))
end_logits = normalizer(torch.from_numpy(end_logits))

start_logits = start_logits.numpy()
end_logits = end_logits.numpy()

In [17]:
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30, n = 1, extension=20):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    
                    re_start_char = max(0,start_char-extension)
                    re_end_char = min(len(context),end_char+extension)
#                     try:
#                         re_start_char = offset_mapping[start_index-extension][0]
#                     except:
#                         re_start_char=0
#                     try:
#                         re_end_char = offset_mapping[end_index+extension][1]
#                     except:
#                         re_end_char=len(context)
#                     print(re_start_char)
                    valid_answers.append(
                        {
                            "score": (start_logits[start_index] + end_logits[end_index])/2,
                            "text": context[start_char: end_char],
#                             "example": example
#                             "feature_index":feature_index,
#                             "features":features[feature_index],
#                             "offset_mapping":offset_mapping,
#                             "context":context,
#                             "start_index":start_index,
#                             "end_index":end_index,
#                             "start_char":start_char,
#                             "end_char":end_char,
#                             "re_start_char":re_start_char,
#                             "re_end_char":re_end_char,
                            "required_text": context[re_start_char: re_end_char]
                        }
                    )
#         print(valid_answers)
        if len(valid_answers) > 0:
            answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)
#             print(answers)
            best_answer = []
            for ans in  answers[:n]:
                best_answer.append(ans)
#             best_answer = answers[:5]
        else:
            best_answer = [""]
        
        predictions[example["id"]] = best_answer#["text"]
        
        
    return predictions

In [18]:
fin_preds = postprocess_qa_predictions(test, test_features, (start_logits, end_logits),
                                       n=5, extension=128)

Post-processing 3804 example predictions split into 17071 features.


In [19]:
# fin_preds

In [20]:
z = list(fin_preds.values())

In [21]:
z[:1]

[[{'score': 0.4730992010815208,
   'text': ' 206',
   'required_text': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறு'},
  {'score': 0.3805177074233963,
   'text': ' 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208',
   'required_text': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக'},
  {'score': 0.29871564877508694,
   'text': ' 208',
   'required_text': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக'},
  {'score': 0.2743517333214154,
   'text': ' 206 (மார்பெலும்பு மூன

In [22]:
questions = test['question'].values
answers = test['answer_text'].values
contexts = test['context'].values

# Postprocess added in this version

In [23]:
bad_starts = [".", ",", "(", ")", "-", "–",  ",", ";"]
bad_endings = ["...", "-", "(", ")", "–", ",", ";"]

tamil_ad = "கி.பி"
tamil_bc = "கி.மு"
tamil_km = "கி.மீ"
hindi_ad = "ई"
hindi_bc = "ई.पू"

def postprocess(pred,context):
    if pred == "":
        return pred
        
    while any([pred.startswith(y) for y in bad_starts]):
        pred = pred[1:]
    while any([pred.endswith(y) for y in bad_endings]):
        if pred.endswith("..."):
            pred = pred[:-3]
        else:
            pred = pred[:-1]
    
    if any([pred.endswith(tamil_ad), pred.endswith(tamil_bc), pred.endswith(tamil_km), pred.endswith(hindi_ad), pred.endswith(hindi_bc)]) and pred+"." in context:
        pred = pred+"."
    
    return pred

In [24]:
def cal_jac(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    mx_score = float(len(c)) / (len(a) + len(b) - len(c))
    return mx_score

In [25]:
final_ans = []
final_ques = []
final_pre = []
final_req_text = []
jac_score =[]
for i, val in enumerate(tqdm(z)):
#     print(i)
    for j in val:
#         print(j)
        pred_text = postprocess(j['text'],contexts[i])
        postprocess_answer = postprocess(answers[i],contexts[i])
        
        final_ans.append(postprocess_answer)
        final_ques.append(questions[i])        
        final_pre.append(pred_text)
        final_req_text.append(j['required_text'])
        jac_score.append(cal_jac(postprocess_answer,pred_text))

100%|██████████| 3804/3804 [00:00<00:00, 16288.63it/s]


In [26]:
final_df = pd.DataFrame()

In [27]:
test.head()

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவு...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் சூரியக் குடும்பம் (Solar Sys...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil


In [28]:
final_df['answer_text'] = final_ans
final_df['question'] = final_ques
final_df['pred_text'] = final_pre
final_df['require_text'] = final_req_text
final_df['jac_score'] = jac_score
final_df.head()

,answer_text,question,pred_text,require_text,jac_score
0,206,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,1.000000
1,206,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட...,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,0.166667
2,206,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,208,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,0.000000
3,206,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட...,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,0.166667
4,206,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட...,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,0.142857


In [29]:
final_df.to_csv('training.csv',index=False)

In [30]:
# final_df

In [31]:
# final_pre